In [ ]:
pip install dash

a.Use any open-source API to access some data in Jason format and then parse the Jason data

In [10]:
import requests
from dash import Dash, html, dcc
from dash.dependencies import Input, Output

# NeoWs API endpoint URL
url = "https://api.nasa.gov/neo/rest/v1/feed"

# API key (replace 'YOUR_API_KEY' with your actual API key)
api_key = "xNm0gIWqBIENxaTxwnDbEPhnb8dN18KPPUNaHhHo"

# Function to fetch NEO data for a given date
def fetch_neo_data(date):
    params = {
        "start_date": date,
        "end_date": date,
        "api_key": api_key
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        if data.get("near_earth_objects"):
            neo_date = min(data["near_earth_objects"].keys())
            closest_neo = min(data["near_earth_objects"][neo_date], key=lambda x: x["close_approach_data"][0]["miss_distance"]["kilometers"])
            return neo_date, closest_neo
        else:
            return None, None
    else:
        return None, None

b.To display the data you may use python library Dash or some other library.

In [11]:
# Create Dash app
app = Dash(__name__)

# Define app layout
app.layout = html.Div([
    html.H1("NeoWs NEO Information"),
    html.Div([
        dcc.Input(
            id='input-date',
            type='text',
            placeholder='Enter Date (YYYY-MM-DD)',
            value='2024-04-15'
        ),
        html.Button('Submit', id='submit-val', n_clicks=0)
    ]),
    html.Div(id='output-data')
])

# Define callback to update output data based on input date
@app.callback(
    Output('output-data', 'children'),
    [Input('submit-val', 'n_clicks')],
    [Input('input-date', 'value')]
)
def update_output(n_clicks, input_date):
    if n_clicks > 0:
        neo_date, closest_neo = fetch_neo_data(input_date)
        if neo_date and closest_neo:
            return html.Div([
                html.Div("Closest NEO on " + neo_date),
                html.Div("Name: " + closest_neo["name"]),
                html.Div("Estimated diameter (meters): " + str(closest_neo["estimated_diameter"]["meters"]["estimated_diameter_min"])),
                html.Div("Relative velocity (km/h): " + str(closest_neo["close_approach_data"][0]["relative_velocity"]["kilometers_per_hour"])),
                html.Div("Miss distance (kilometers): " + str(closest_neo["close_approach_data"][0]["miss_distance"]["kilometers"]))
            ])
        else:
            return html.Div("No NEO data available for the selected date.")
    else:
        return html.Div()

if __name__ == '__main__':
    app.run_server(debug=True)


<IPython.core.display.Javascript object>